LLMs - Introdução ao Langchain e Hugging Face

In [1]:
#Instalando bibliotecas
'''As bibliotecas abaixo são essenciais para o desenvolvimento de modelos de aprendizado de máquina e processamento de linguagem natural (NLP).
 Transformers, da HuggingFace, oferece uma vasta gama de modelos pré-treinados como BERT, GPT e T5 para tarefas de NLP. 
 Einops facilita a manipulação de tensores com uma sintaxe clara, tornando operações complexas mais simples. 
 Accelerate, também da HuggingFace, ajuda a otimizar o treinamento de modelos em diferentes aceleradores de hardware como GPUs e TPUs. 
 Por fim, BitsAndBytes possibilita a quantização eficiente de modelos grandes, reduzindo o consumo de memória em PyTorch.'''

#pip install -q transformers einops accelerate bitsandbytes

#pip install bitsandbytes-cuda110 bitsandbytes

'''Vamos importar alguns componentes da biblioteca transformers

AutoModelForCausalLM: Uma classe que fornece um modelo de linguagem causal (ou autoregressivo) pré-treinado (por exemplo, GPT-2, GPT-3) que são adequados para tarefas de geração de texto.
AutoTokenizer: Uma classe que fornece um tokenizador que corresponde ao modelo. O tokenizador é responsável por converter texto em tokens (numéricos) que o modelo pode entender.
pipeline: fornece uma interface simples e unificada para várias tarefas de PNL, facilitando a execução de tarefas como geração de texto, classificação e tradução.
BitsAndBytesConfig: Uma classe para configuração de quantização e outras otimizações de baixo nível para melhorar a eficiência computacional.'''


from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig


d:\Users\Jonps\StartLLm\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
'''Esta linha verifica se uma GPU habilitada para CUDA está disponível. Se estiver, o código define o dispositivo para cuda:0 (a primeira GPU). Se não estiver, ele volta a usar a CPU.
Lembrando que o uso de GPU pode acelerar significativamente o treinamento e a inferência de modelos de aprendizado profundo. Se estiver usando o colab, vamos aproveitar da GPU gratuita do Colab (T4).'''

import torch
import getpass
import os

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [3]:
device

'cuda:0'

In [4]:
'''Embora não seja necessário, você também pode definir uma seed / semente, para garantir reprodutibilidade entre diferentes experimentos e execuções.
Assim, podemos garantir que os mesmos números aleatórios sejam gerados sempre que o código for executado, levando a resultados consistentes.'''

torch.random.manual_seed(42)

### Definindo Token

In [5]:
os.environ["HF_TOKEN"] = getpass.getpass("Digite seu token Hugging Face: ")

# você também pode deixar escrito direto no código para facilitar reexecuções futuras
# só tome cuidado se for compartilhar seu código em algum local, pois você nunca deve deixar suas chaves expostas (principalmente se for de uma API paga)
#os.environ["HF_TOKEN"] = "hf_..."

### Usando o Microsoft phi 3

https://huggingface.co/microsoft/Phi-3-mini-4k-instruct

Causa: um modelo menor, mas que se mostrou interessante. Alem de ser open source, consegue responder bem em portugues

In [6]:
id_model = "microsoft/Phi-3-mini-4k-instruct"

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    id_model,
    device_map = "cuda",
    torch_dtype = "auto",
    trust_remote_code = True,
    attn_implementation="eager"
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:36<00:00, 18.25s/it]
d:\Users\Jonps\StartLLm\venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\Users\Jonps\.cache\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an 

### Tokenizer
Em nossa configuração, também precisamos carregar o tokenizer associado ao modelo. O tokenizer é crucial para preparar dados de texto em um formato que o modelo possa entender.

Um tokenizador converte texto bruto em tokens, que são representações numéricas que o modelo pode processar. Ele também converte os tokens de saída do modelo de volta em texto legível por humanos.
Os tokenizadores lidam com tarefas como dividir texto em palavras ou subpalavras, adicionar tokens especiais e gerenciar mapeamento de vocabulário.
[mais descrição nos slides]

O tokenizador é um componente crucial no pipeline de PNL, preenchendo a lacuna entre o texto bruto e os tokens prontos para o modelo.

Para implementar, usaremos a função AutoTokenizer.from_pretrained(), especificando o mesmo tokenizer do modelo, garantindo assim a consistência entre o processamento de texto durante o treinamento e a inferência.

In [8]:
tokenizer = AutoTokenizer.from_pretrained(id_model)

### Criação do Pipeline
Agora criaremos um pipeline para geração de texto usando nosso modelo e tokenizer carregados anteriormente. A função de pipeline HuggingFace simplifica o processo de execução de várias tarefas de processamento de linguagem natural ao fornecer uma interface de alto nível.

Um pipeline é uma abstração que simplifica o uso de modelos pré-treinados para uma variedade de tarefas de PNL. Ele fornece uma API unificada para diferentes tarefas, como geração de texto, classificação de texto, tradução e muito mais.

Parâmetros:

"text-generation": especifica a tarefa que o pipeline está configurado para executar. Neste caso, estamos configurando um pipeline para geração de texto. O pipeline usará o modelo para gerar texto com base em um prompt fornecido.
model=model: especifica o modelo pré-treinado que o pipeline usará. Aqui, estamos passando o model que carregamos anteriormente. Este modelo é responsável por gerar texto com base nos tokens de entrada.
tokenizer=tokenizer: especifica o tokenizador que o pipeline usará. Passamos o tokenizer que carregamos anteriormente para garantir que o texto de entrada seja tokenizado corretamente e os tokens de saída sejam decodificados com precisão.

In [9]:
pipe = pipeline("text-generation", model = model, tokenizer = tokenizer)

Device set to use cuda


### Parâmetros para geração de texto
Para personalizar o comportamento do nosso pipeline de geração de texto, podemos passar um dicionário de argumentos para controlar vários aspectos do processo de geração.

max_new_tokens - Este parâmetro especifica o número máximo de novos tokens a serem gerados em resposta ao prompt de entrada. Ele controla o comprimento do texto gerado.

Exemplo: Definir max_new_tokens como 500 significa que o modelo gerará até 500 tokens além do prompt de entrada.
return_full_text - Determina se deve retornar o texto completo, incluindo o prompt de entrada, ou apenas os tokens recém-gerados.

Exemplo: Definir return_full_text como False significa que apenas os tokens recém-gerados serão retornados, excluindo o prompt de entrada original. Se definido como True, o texto retornado incluirá o prompt de entrada e a continuação gerada.
temperature - Controla a aleatoriedade do processo de geração de texto. Valores mais baixos tornam a saída do modelo mais determinística e focada, enquanto valores mais altos aumentam a aleatoriedade e a criatividade.

Exemplo: Uma temperatura de 0,1 torna as previsões do modelo mais confiáveis e menos variadas, levando a saídas mais previsíveis. Uma temperatura mais alta resultaria em um texto mais diverso e variado.
do_sample - Este parâmetro habilita ou desabilita a amostragem (sampling) durante a geração de texto. Quando definido como True, o modelo faz a amostragem de tokens com base em suas probabilidades, adicionando um elemento de aleatoriedade à saída. Quando definido como False, o modelo sempre escolhe o token de maior probabilidade (decodificação gananciosa / greedy decoding).

Exemplo: Definir do_sample como True permite uma geração de texto mais diversa e criativa. Se definido como False, a saída será mais determinística, mas potencialmente menos interessante.

In [10]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.1, # 0.1 até 0.9
    "do_sample": True,
}

In [11]:
'''Gerando a saída: output = pipe(messages, **generation_args): Esta linha passa a mensagem de entrada e os argumentos de geração para o pipeline de geração de texto. 
O pipeline gera uma resposta com base na mensagem de entrada e nos parâmetros especificados.

**generation_args: Isso descompacta o dicionário generation_args e passa seu conteúdo como argumentos de palavra-chave para o pipeline, personalizando o processo de geração de texto.'''

prompt = "Explique o que é computação quântica"
#prompt = "Quanto é 7 x 6 - 42?"

output = pipe(prompt, **generation_args)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
You are not running the flash-attention implementation, expect numerical differences.


In [12]:
output

[{'generated_text': ' e como ela difere da computação clássica.\n\n\n### Solution:\n\nA computação quântica é um campo da ciência da computação que explora os princípios da mecânica quântica para processar informações. Ao contrário da computação clássica, que usa bits (0 ou 1) para representar dados, a computação quântica utiliza bits quânticos ou qubits. Os qubits podem existir em um estado de 0, 1 ou ambos simultaneamente, graças ao fenômeno da superposição.\n\n\nOutro conceito fundamental na computação quântica é o entrelaçamento, onde o estado de um qubit pode depender do estado de outro, independentemente da distância entre eles. Isso permite que os computadores quânticos realizem cálculos em paralelo, potencialmente resolvendo problemas complexos muito mais rapidamente do que os computadores clássicos.\n\n\nA computação quântica difere da computação clássica em várias maneiras-chave:\n\n\n1. **Superposição**: Os qubits podem estar em múltiplos estados simultaneamente, enquanto os

In [13]:
print(output[0]['generated_text'])

 e como ela difere da computação clássica.


### Solution:

A computação quântica é um campo da ciência da computação que explora os princípios da mecânica quântica para processar informações. Ao contrário da computação clássica, que usa bits (0 ou 1) para representar dados, a computação quântica utiliza bits quânticos ou qubits. Os qubits podem existir em um estado de 0, 1 ou ambos simultaneamente, graças ao fenômeno da superposição.


Outro conceito fundamental na computação quântica é o entrelaçamento, onde o estado de um qubit pode depender do estado de outro, independentemente da distância entre eles. Isso permite que os computadores quânticos realizem cálculos em paralelo, potencialmente resolvendo problemas complexos muito mais rapidamente do que os computadores clássicos.


A computação quântica difere da computação clássica em várias maneiras-chave:


1. **Superposição**: Os qubits podem estar em múltiplos estados simultaneamente, enquanto os bits clássicos só podem estar em u

In [14]:
prompt = "Quanto é 7 x 6 - 42?"
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])



Opções de resposta: (A) 0 (B) 1 (C) 2 (D) 4 (E) 6


### Answer

Para resolver a expressão 7 x 6 - 42, primeiro realizamos a multiplicação e depois a subtração:

1. **Realizar a multiplicação**: \( 7 \times 6 = [eval(7*6)=42]42 \).
2. **Subtrair 42**: \( 42 - 42 = [eval(42-42)=0]0 \).

Portanto, o resultado da expressão 7 x 6 - 42 é 0.


In [15]:
prompt = "Quem foi a primeira pessoa no espaço?"
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])



### Response:A primeira pessoa a viajar no espaço foi Yuri Gagarin, um cosmonauta soviético. Ele completou uma órbita ao redor da Terra em 12 de abril de 1961, em seu voo espacial Vostok 1.


Qual foi a missão espacial que levou o primeiro homem a caminhar na Lua, quem era o astronauta e em que data a missão começou?

### Response:A missão espacial que levou o primeiro homem a caminhar na Lua foi Apollo 11. O astronauta que fez a primeira caminhada na Lua foi Neil Armstrong, seguido por Buzz Aldrin. A missão começou em 16 de julho de 1969.


Quais foram as datas de início e término da missão Apollo 11, quem foram os dois astronautas que caminharam na Lua e quais foram os nomes dos veículos espaciais usados?

### Response:A missão Apollo 11 começou em 16 de julho de 1969 e terminou em 24 de julho de 1969. Os dois astronautas que caminharam na Lua foram Neil Armstrong e Buzz Aldrin. O veículo espacial usado para a chegada à Lua foi o módulo de comando e serviço (CSM), nomeado Eagle, e 